In [12]:
import warnings
warnings.filterwarnings("ignore")
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.mol_structure import draw_one_mol, draw_graphs
from evaluation.new_structural_metric import symmetric_graph_set_distance,atom,cycle,neighborhood
from evaluate import evaluate
#import utils.graph_generators as gen
import torch
import networkx as nx
import os 
import sys
import json
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)
import rdkit
import numpy as np
from evaluation.utils import get_data,get_graph_data,  get_mock_data,remove_empty_graphs_and_targets,\
    get_generated_data,preprocess,get_generated_graph_data,_preprocess



# Rewired

In [3]:
import pandas as pd
def get_rewired_graph_data(name,percent):
    generated_neg_graphs=pd.read_pickle(f'data/rewired/{name}_train2_neg/{name}_train2_neg_rewired_with_{percent}.pkl')
    generated_pos_graphs=pd.read_pickle(f'data/rewired/{name}_train2_pos/{name}_train2_pos_rewired_with_{percent}.pkl')
    generated_graphs,generated_targets = generated_pos_graphs + generated_neg_graphs, [1]*len(generated_pos_graphs)+[0]*len(generated_neg_graphs)
    return  generated_graphs, np.array(generated_targets)

In [4]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn','structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=['5','10','25','33','40','50','66','75','90','100']
metrics={}
for generator in ['rewired']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_rewired_graph_data(dataset_name+split,percent)
                
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_1_50
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Cannot compute these structural metrics
Now computing the auc_roc based  metric


# Pertubed with ER

In [3]:
import pandas as pd
def get_corrupted_graph_data(name,percent):
    generated_neg_graphs=pd.read_pickle(f'data/corrupted/{name}_train2_neg/{name}_train2_neg_perturbed_with_{percent}.pkl')
    generated_pos_graphs=pd.read_pickle(f'data/corrupted/{name}_train2_pos/{name}_train2_pos_perturbed_with_{percent}.pkl')
    generated_graphs,generated_targets = generated_pos_graphs + generated_neg_graphs, [1]*len(generated_pos_graphs)+[0]*len(generated_neg_graphs)
    return  generated_graphs, np.array(generated_targets)

In [7]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=['5','10','25','33','40','50','66','75','90','100']
metrics={}
for generator in ['corrupted']:
    for dataset_name in ['synthetic_1','synthetic_1']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_corrupted_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_1_50
Now computing classifier based metrics
Cannot compute the classifier based metrics
Now computing structural based metrics
Cannot compute these structural metrics
Now computing the auc_roc based  metric


# Mode Dropping


In [24]:
import pandas as pd
def get_mode_dropping_graph_data(name,percent):
    #try:
   generated_graphs=pd.read_pickle(f'data/mode_dropping/{name}/{name}_no_of_clusters_dropped_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
        target= int(g.nodes[0]['target'])
        targets.append(target)
   return  generated_graphs, np.array(targets)

In [26]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(1, 32)]
metrics={}
for generator in ['mode_dropping']:
    for dataset_name in ['synthetic_1']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_dropping_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)



synthetic_1_50
Now computing classifier based metrics
Cannot compute the classifier based metrics
Now computing structural based metrics


In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(1, 33)]
metrics={}
for generator in ['mode_dropping']:
    for dataset_name in ['synthetic_2']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_dropping_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

# Mode Collapse

In [10]:

import pandas as pd
def get_mode_collapse_graph_data(name,percent):
    #try:
   generated_graphs=pd.read_pickle(f'data/mode_collapse/{name}/{name}_no_of_clusters_collapsed_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
        target= int(g.nodes[0]['target'])
        targets.append(target)
   return  generated_graphs, np.array(targets)

In [11]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(1, 32)]
metrics={}
for generator in ['mode_collapse']:
    for dataset_name in ['synthetic_1']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_collapse_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_1_50
Now computing classifier based metrics
Cannot compute the classifier based metrics
Now computing structural based metrics
Cannot compute these structural metrics
Now computing the auc_roc based  metric


In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(1, 33)]
metrics={}
for generator in ['mode_collapse']:
    for dataset_name in ['synthetic_2']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_collapse_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

# Additionally train1 versus train2

In [4]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(1, 33)]
metrics={}
for generator in ['train2']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            print(dataset_name+split)
            train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
            generated_graphs, generated_targets= train2_graphs, train2_targets
            splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
            metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            metrics['degree_of_perturbation']=percent
            print(metrics)

synthetic_2_50
Now computing classifier based metrics
Cannot compute the classifier based metrics
Now computing structural based metrics
Cannot compute these structural metrics
Now computing the auc_roc based  metric


# SWINGNN as we increase the number of training epochs

In [12]:

import pandas as pd
from glob import glob
from sklearn.utils import shuffle                                                                                                                                                                                                                                                                                                                                                                                                                                                              
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.utils import remove_empty_graphs_and_targets
import numpy as np
def get_swingnn_graph_data(name,percent,split,path='data/SWINGNN_experiments/'):
     path+=f'{percent}-percent/'
     path_postives=path+r'{}/{}/pos/*.pkl'.format(name,name+split)
     files = glob(path_postives)
     generated_pos_graphs=pd.read_pickle(files[0])
     path_negatives=path+r'{}/{}/neg/*.pkl'.format(name,name+split)
     files = glob(path_negatives)
     generated_neg_graphs=pd.read_pickle(files[0])
     generated_graphs,generated_targets = generated_pos_graphs + generated_neg_graphs, [1]*len(generated_pos_graphs)+[0]*len(generated_neg_graphs)

     return  generated_graphs, np.array(generated_targets)

In [ ]:
name='synthetic_1'
percent='100'
split='_50'
get_swingnn_graph_data(name,percent,split)

In [2]:
from glob import glob
from sklearn.utils import shuffle                                                                                                                                                                                                                                                                                                                                                                                                                                                              
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.utils import remove_empty_graphs_and_targets
import numpy as np

def get_swingnn_molecular_data(name,percent, path='data/SWINGNN_experiments/', generator_name='stgg',return_smiles=False):
     path+=f'{percent}-percent/'
     pos_list, neg_list=[],[]
     path_postives=path+r'{}/{}/pos/*.txt'.format(name,name+'_50')
     files = glob(path_postives)
     
     with open(files[0]) as my_file:
          for line in my_file:
               pos_list.append(line.strip())

     path_negatives=path+r'{}/{}/neg/*.txt'.format(name,name+'_50')
     files = glob(path_negatives)
     
     with open(files[0]) as my_file:
          for line in my_file:
               neg_list.append(line.strip())

     pos_graphs=list_of_smiles_to_nx_graphs(pos_list)
     neg_graphs=list_of_smiles_to_nx_graphs(neg_list)
     #print(len(neg_graphs))
     generated_graphs,generated_targets = pos_graphs + neg_graphs, [1]*len(pos_graphs)+[0]*len(neg_graphs)
     #generated_graphs, generated_targets = shuffle(generated_graphs, generated_targets)
     generated_graphs, generated_targets = remove_empty_graphs_and_targets(generated_graphs, generated_targets)
     if return_smiles:
          return  generated_graphs, generated_targets,pos_list+neg_list
     return  generated_graphs, np.array(generated_targets)

In [8]:
name='ames'
percent='100'
split='_50'
_,generated_targets, generated_smiles=get_swingnn_molecular_data(name,percent,return_smiles=True) 

In [10]:
generated_targets[-1]

0

## Molecular graphs

### nn and structural 

In [111]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_25','_33','_40','_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['swingnn']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
split='_50'
for generator in generators:
    for dataset_name in current_smiles_datasets:
             for percent in experiments:
                print(dataset_name+split)
                train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                generated_graphs, generated_targets=get_swingnn_molecular_data(dataset_name,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100
                print(metrics)

ames_50
['data/SWINGNN_experiments/5-percent/ames/ames_50/pos\\gen_smiles.txt']
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Cannot compute these structural metrics
{'activations_time': 0.9547960758209229, 'fid': 33.27688554415545, 'fid_time': 0.966336727142334, 'kid': 0.92367905, 'kid_time': 47.91331601142883, 'precision': 0.900667451904201, 'recall': 0.6486062033765214, 'f1_pr': 0.7541423633017976, 'precision_time': 1.9263620376586914, 'recall_time': 1.9263620376586914, 'f1_pr_time': 1.9263620376586914, 'density': 0.7696113074204947, 'coverage': 0.3541421279937181, 'f1_dc': 0.4850853653998528, 'density_time': 1.5020718574523926, 'coverage_time': 1.5020718574523926, 'f1_dc_time': 1.5020718574523926, 'mmd_rbf': 0.18187773

#### train2

In [3]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train2']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
split='_50'
for generator in generators:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            print(dataset_name+split)
            train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
            generated_graphs, generated_targets=train2_graphs , train2_targets
            splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
            metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_25
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Cannot compute these structural metrics
{'activations_time': 0.41626691818237305, 'fid': 13.457352981320554, 'fid_time': 0.42125964164733887, 'kid': -1.0777628, 'kid_time': 6.520142555236816, 'precision': 0.9646504320502749, 'recall': 0.9795758051846033, 'f1_pr': 0.97206582989574, 'precision_time': 0.7290303707122803, 'recall_time': 0.7290303707122803, 'f1_pr_time': 0.7290303707122803, 'density': 0.9558523173605655, 'coverage': 0.9701492537313433, 'f1_dc': 0.962957722176193, 'density_time': 0.6316354274749756, 'coverage_time': 0.6316354274749756, 'f1_dc_time': 0.6316354274749756, 'mmd_rbf': 0.002143561840057373, 'mmd_rbf_time': 1.0874826908111572, 'mmd_linear': 0.157119

### molecular

In [ ]:
from evaluation.moses.metrics.metrics import get_all_metrics
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split['_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['swingnn']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            for percent in experiments:
                print(dataset_name+split)
                graphs, splits =get_data(dataset_name+split, return_smiles=True)
                train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
                _,generated_targets, generated_smiles=get_swingnn_molecular_data(name,percent,return_smiles=True) 
                metrics=get_all_metrics(gen=generated_smiles,train=train1_smiles,test=train1_smiles)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100 
                print(metrics)

#### train2


In [24]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split ='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train2']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            print(dataset_name+split)
            graphs, splits =get_data(dataset_name+split, return_smiles=True)
            train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
            train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
            metrics=get_all_metrics(gen=train2_smiles,train=train1_smiles,test=train1_smiles)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.999, 'unique@10000': 0.9988221436984688, 'FCD/Test': 0.8156919603441324, 'SNN/Test': 0.5756633373852753, 'Frag/Test': 0.9878820451873492, 'Scaf/Test': 0.8593618010840082, 'FCD/TestSF': 22.184285088577205, 'SNN/TestSF': 0.33677946461981123, 'Frag/TestSF': -0.02581333437055111, 'Scaf/TestSF': 0.20333785426573803, 'IntDiv': 0.9102053951183938, 'IntDiv2': 0.8921645781772736, 'Filters': 0.5618374558303887, 'logP': 0.08376508833922258, 'SA': 0.04752492131394497, 'QED': 0.0039007607322941876, 'weight': 4.032424028268554, 'Novelty': 0.9976415094339622, 'dataset': 'ames_50', 'generator_name': 'swingnn', 'numer_of_epochs_trained': 250.0}
bbb_martins_50
{'valid': 1.0, 'unique@1000': 0.9957805907172996, 'unique@10000': 0.9957805907172996, 'FCD/Test': 3.6841985920182054, 'SNN/Test': 0.5064036945397173, 'Frag/Test': 0.966294915338154, 'Scaf/Test': 0.5502519513284054, 'FCD/TestSF': 20.86298113850406, 'SNN/TestSF': 0.3738190557404242, 'Frag/TestSF': 0.0617232322

KeyboardInterrupt: 

## Synthetic graphs 

In [18]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn','structural', 'auc_roc']
generators=['swingnn']
splits=['_25','_33','_40','_50']
experiments=[5,10,25,33,40,50,66,75,100]
metrics={}
for generator in ['swingnn']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            for percent in experiments:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_swingnn_graph_data(dataset_name,percent,split)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100
                print(metrics)

synthetic_1_25
data/SWINGNN_experiments/5-percent/synthetic_1/synthetic_1_25/pos/*.pkl
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Cannot compute these structural metrics
Now computing the auc_roc based  metric
Error when computing AUC_ROC with NSPDK
{'activations_time': 0.09308505058288574, 'fid': 1992.090677079746, 'fid_time': 0.09808611869812012, 'kid': 32063.271, 'kid_time': 44.243937969207764, 'precision': 0.01485148514851485, 'recall': 0.025, 'f1_pr': 0.018644188556302177, 'precision_time': 0.16258978843688965, 'recall_time': 0.16258978843688965, 'f1_pr_time': 0.16258978843688965, 'density': 0.0029702970297029703, 'coverage': 0.005, 'f1_dc': 0.0037373549602240336, 'density_time': 0.1354506015777588, 'coverage_time'

In [4]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn','structural', 'auc_roc']
generators=['train2']
splits=['_25','_33','_40','_50']
experiments=[5,10,25,33,40,50,66,75,100]
metrics={}
for generator in ['train2']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            print(dataset_name+split)
            train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
            generated_graphs, generated_targets=train2_graphs,train2_targets
            splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
            metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)


synthetic_1_25
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Cannot compute these structural metrics
Now computing the auc_roc based  metric
